In [1]:
import tensorflow as tf
print(tf.__version__)

2.5.0


In [2]:
# save_path = 'C:/Users/nakul/OneDrive/Desktop/dl projects'
model_name = 'xray_model'
# model_path = save_path + '/' + model_name

from sklearn.metrics import confusion_matrix
import numpy as np

def binary_balanced_accuracy(y_true, y_pred):
    y_true = y_true.numpy()
    y_pred = y_pred.numpy()

    y_true = y_true.ravel()
    y_pred = np.round(y_pred.ravel())
    num_classes = len(np.unique(y_true))

    cm = confusion_matrix(y_true, y_pred).T
    balanced_accuracy = 0
    for i in range(num_classes):
        num = cm[i,i]
        den = np.sum(cm[:,i])
        if num == 0 :
            acc = 0
        else:
            acc = num / den
        balanced_accuracy += acc

    return (balanced_accuracy / num_classes)

model = tf.keras.models.load_model(model_name , custom_objects = {'binary_balanced_accuracy' : binary_balanced_accuracy})

In [3]:
import joblib
labels = joblib.load('labels.joblib')

print(labels)

{0: 'NORMAL', 1: 'PNEUMONIA'}


In [4]:
from pywebio.input import file_upload
from pywebio.output import put_image,put_tabs

In [5]:
def predict_class():
    img = file_upload(placeholder = 'Upload image of the xray')
    x = tf.io.decode_image(img['content'])
    try:
        x = tf.image.rgb_to_grayscale(x)
    except :
        pass
    x = tf.image.resize(x,[150,150])
    x = x / 255.0
    x = tf.expand_dims(x, axis=0)
    cls = model.predict(x)
    cls = np.round(cls.ravel())
    text = 'Predicted Class : ' + str(labels[int(cls[0])])
    put_tabs([
    {'title': 'Result', 'content': text},
    {'title': 'Uploaded Image', 'content': [
        put_image(img['content'],width='300px',height='300px',title='X-ray_Image')
    ]},
    ])

In [6]:
predict_class()